In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-28 17:23:47--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-28 17:23:48 (24.7 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [3]:
import minsearch

In [4]:
import json

In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
#docs_raw

In [7]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [10]:
q = 'the course has already started, can I still enroll?'

In [11]:
index.fit(documents)

In [12]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [13]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [14]:
from openai import OpenAI

In [15]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [16]:
q

'the course has already started, can I still enroll?'

In [17]:
response = client.chat.completions.create(
    model='phi3',
    messages=[{"role": "user", "content": q}]
)

In [18]:
response

ChatCompletion(id='chatcmpl-527', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=" Whether or not you can still enroll in a course that has already started depends on several factors. These factors include:\n\n1. The policies of the institution offering the course: Some institutions have strict cutoff dates for enrolling in courses after they've begun, while others may allow late admissions based on specific criteria.\n\n2. Available classroom space and resources: If there are still seats available or you can catch up quickly with online resources to keep pace with other students, it may be possible to enroll even after the course has started.\n\n3. Course duration and workload: Consider if the remaining time in the course is sufficient for you to complete required assignments and participate meaningfully in class discussions or group projects. Enrolling later might make this challenging.\n\n4. Instructor's discretion: Some instructo

In [19]:
response.choices[0].message.content

" Whether or not you can still enroll in a course that has already started depends on several factors. These factors include:\n\n1. The policies of the institution offering the course: Some institutions have strict cutoff dates for enrolling in courses after they've begun, while others may allow late admissions based on specific criteria.\n\n2. Available classroom space and resources: If there are still seats available or you can catch up quickly with online resources to keep pace with other students, it may be possible to enroll even after the course has started.\n\n3. Course duration and workload: Consider if the remaining time in the course is sufficient for you to complete required assignments and participate meaningfully in class discussions or group projects. Enrolling later might make this challenging.\n\n4. Instructor's discretion: Some instructors may allow late enrollment, while others prefer students to join at the start of the semester for a cohesive learning experience wit

In [25]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [23]:
context = ""
    
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [28]:
prompt = prompt_template.format(question=q, context=context).strip()

In [30]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fi

In [32]:
response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
)
    
response.choices[0].message.content

" Yes, even if you don't register, you're still eligible to submit homeworks after the course starts on January 15th, 2024 at 17h00. However, be aware of deadlines for turning in final projects and avoid leaving everything until the last minute.\n\nAdditionally, all materials will be available after the course finishes, allowing you to continue at your own pace or prepare for the next cohort. You can also start working on your final capstone project after enrolling."

In [33]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [34]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [35]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [36]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [37]:
rag(query)

" To run Kafka in the terminal (assuming you are referring to a Java application using Kafka, as there's no direct Python `producer.py` reference or Docker `build.sh` in the context given), follow this command based on the provided context:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nRemember to replace `<jar_name>` with your actual Kafka JAR file name without the version suffix."

In [38]:
rag('the course has already started, can I still enroll?')

" Yes, even if you don't register beforehand, you can still enroll in the course after it has started and participate by submitting homeworks and final projects, provided that there are deadlines for these assignments. The course materials will also be available for self-pacing afterwards.\n\nHowever, please note that the course officially starts on January 15th at 17:00. It is highly recommended to register beforehand using the provided link and join relevant communication channels such as Google Calendar, Telegram, DataTalks.Club's Slack channel for announcements and support during self-paced mode."

In [39]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [40]:
from elasticsearch import Elasticsearch

In [41]:
es_client = Elasticsearch('http://localhost:9200') 

In [42]:
es_client.info()

ObjectApiResponse({'name': '0fec609160d5', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Q8O2hCpxS6enm92slMd_EA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [43]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [44]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [45]:
from tqdm.auto import tqdm

In [46]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [47]:
query = 'I just disovered the course. Can I still join it?'

In [48]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [49]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [50]:
rag(query)

' Yes, you can still join the course even if it has started. However, be aware of deadlines for turning in final projects. Additionally, after the course finishes, all materials will remain available and you can follow the course at your own pace. You can also continue working on homework and preparing for the next cohort during this time.'